In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
## Copy contents from My Drive to "/content" in order to import all scripts.
!cp -r /content/drive/My\ Drive/SC_RL /content

In [3]:
#rm -rf SC_RL/

**Goal of this Project:** Maintain inventory of all products while minimizing operational costs.

In [4]:
!pip install import_ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=3a462e70f9e1ff7c70ee8a087cde4416642ee95b6a96e0b10eb91af474cb59bc
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [5]:
## Import libraries.
import import_ipynb
import numpy as np
import os
import tensorflow as tf

from pathlib import Path

from SC_RL.Environments import warehouse_store

importing Jupyter notebook from /content/SC_RL/Environments/warehouse_store.ipynb


In [6]:
metadata_file = Path(os.getcwd()+"/SC_RL/data/instacart-market-basket-analysis/products_metadata.xlsx")
forecast_data = Path(os.getcwd()+"/SC_RL/data/instacart-market-basket-analysis/scenarios.xlsx")
w = warehouse_store.warehouse_store()
warehouse_item_limit = 500

In [7]:
def reward_function(states, actions, quantity_removed, timestep):
  '''
  Should ensure that inventory is stocked, but at the same time, ensure that
  wastage is minimized.
  1 - (quantity_restocked/total quantity of products)
    - (quantity of expired products/total quantity of products)
  '''
  ## only check quantity portion when it is thrown away.
  p_restocked = np.ndarray.sum(actions)
  q_max = np.ndarray.sum(states[:,0])
  reward = 1 - (p_restocked+quantity_removed)/q_max
  return reward

In [8]:
## Test passing a function as argument here:
total_reward = w.simulate(metadata_file,forecast_data,reward_function) # Need not initialize; just simulate.
print(total_reward)

-292.4123141298442


## **Actor-Critic**

In [ ]:
## Parameters:
episodes = 100
GAMMA = 0.9     # reward discount in TD error
LR_A = 0.001    # learning rate for actor
LR_C = 0.01     # learning rate for critic

In [ ]:
## Actor-Critic Methods:
class Actor(object):
    def __init__(self, sess, n_features, n_actions, lr=0.001):
        self.sess = sess

        self.s = tf.placeholder(tf.float32, [1, n_features], "state")
        self.a = tf.placeholder(tf.int32, None, "act")
        self.td_error = tf.placeholder(tf.float32, None, "td_error")  # TD_error

        with tf.variable_scope('Actor'):
            l1 = tf.layers.dense(
                inputs=self.s,
                units=20,    # number of hidden units
                activation=tf.nn.relu,
                kernel_initializer=tf.random_normal_initializer(0., .1),    # weights
                bias_initializer=tf.constant_initializer(0.1),  # biases
                name='l1'
            )

            self.acts_prob = tf.layers.dense(
                inputs=l1,
                units=n_actions,    # output units
                activation=tf.nn.softmax,   # get action probabilities
                kernel_initializer=tf.random_normal_initializer(0., .1),  # weights
                bias_initializer=tf.constant_initializer(0.1),  # biases
                name='acts_prob'
            )

        with tf.variable_scope('exp_v'):
            log_prob = tf.log(self.acts_prob[0, self.a])
            self.exp_v = tf.reduce_mean(log_prob * self.td_error)  # advantage (TD_error) guided loss

        with tf.variable_scope('train'):
            self.train_op = tf.train.AdamOptimizer(lr).minimize(-self.exp_v)  # minimize(-exp_v) = maximize(exp_v)

    def learn(self, s, a, td):
        s = s[np.newaxis, :]
        feed_dict = {self.s: s, self.a: a, self.td_error: td}
        _, exp_v = self.sess.run([self.train_op, self.exp_v], feed_dict)
        return exp_v

    def choose_action(self, s):
        s = s[np.newaxis, :]
        probs = self.sess.run(self.acts_prob, {self.s: s})   # get probabilities for all actions
        return np.random.choice(np.arange(probs.shape[1]), p=probs.ravel())   # return a int


class Critic(object):
    def __init__(self, sess, n_features, lr=0.01):
        self.sess = sess

        self.s = tf.placeholder(tf.float32, [1, n_features], "state")
        self.v_ = tf.placeholder(tf.float32, [1, 1], "v_next")
        self.r = tf.placeholder(tf.float32, None, 'r')

        with tf.variable_scope('Critic'):
            l1 = tf.layers.dense(
                inputs=self.s,
                units=20,  # number of hidden units
                activation=tf.nn.relu,  # None
                # have to be linear to make sure the convergence of actor.
                # But linear approximator seems hardly learns the correct Q.
                kernel_initializer=tf.random_normal_initializer(0., .1),  # weights
                bias_initializer=tf.constant_initializer(0.1),  # biases
                name='l1'
            )

            self.v = tf.layers.dense(
                inputs=l1,
                units=1,  # output units
                activation=None,
                kernel_initializer=tf.random_normal_initializer(0., .1),  # weights
                bias_initializer=tf.constant_initializer(0.1),  # biases
                name='V'
            )

        with tf.variable_scope('squared_TD_error'):
            self.td_error = self.r + GAMMA * self.v_ - self.v
            self.loss = tf.square(self.td_error)    # TD_error = (r+gamma*V_next) - V_eval
        with tf.variable_scope('train'):
            self.train_op = tf.train.AdamOptimizer(lr).minimize(self.loss)

    def learn(self, s, r, s_):
        s, s_ = s[np.newaxis, :], s_[np.newaxis, :]

        v_ = self.sess.run(self.v, {self.s: s_})
        td_error, _ = self.sess.run([self.td_error, self.train_op],
                                          {self.s: s, self.v_: v_, self.r: r})
        return td_error

In [ ]:
## Initialize session:
sess = tf.Session()

actor = Actor(sess, n_features=N_F, n_actions=N_A, lr=LR_A)
critic = Critic(sess, n_features=N_F, lr=LR_C)     # we need a good teacher, so the teacher should learn faster than the actor

sess.run(tf.global_variables_initializer())

tf.summary.FileWriter("logs/", sess.graph)

In [ ]:
## Training:
for ep in range(episodes):
  ## Run environment here.
  w.simulate(metadata_file,forecast_data,)
  '''Plot reward here.'''
  pass

In [ ]:
## Evaluate: